In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero
dir_ = '../../../data'
group_dir_ = os.path.join(dir_, 'groups/high')

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

num_user = len(svd['uid'].unique())
num_user

220

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '20']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
pop_dict = {}
for i in df['tid'].unique():
    pop_dict[i] = len(df[df['tid']==i])/num_user

In [5]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,3,2.544071,1
1,0,7,2.545822,1
2,0,18,2.152490,1
3,0,20,1.882396,1
4,0,31,2.410806,1


In [6]:
# Count POP to DF
tid_list = []
pop_list = []
for i in df['tid'].unique():
    tid_list.append(i)
    pop_list.append(len(df[df['tid']==i])/num_user*5)
    
d = {'tid': tid_list, 'rating': pop_list}
df_pop = pd.DataFrame(data=d)
df_pop = df_pop.sort_values(by=['rating'], ascending=False)
df_pop[:10]

,tid,rating
166,166,3.840909
7209,7209,3.295455
350,350,3.113636
3761,3761,3.045455
80,80,3.022727
1017,1017,2.840909
1753,1753,2.750000
428,428,2.727273
200,200,2.681818
3159,3159,2.659091


In [7]:
top_percent = 0.01
long_tail = df_pop[int(len(df_pop)*top_percent):]
long_tail = long_tail['tid'].unique()
len(long_tail), len(df_pop), len(df_pop)-len(long_tail)

(54420, 54969, 549)

In [8]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [ ]:
for lambda_ in tqdm(lambdas):
    top_n_size = 4
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    threshold = 0.0
    avg_pops = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        pops = []
        for j in (range(len(groups))):
            group = groups[j]
            top_n_list = top_n_lists[j]
            for k in range(len(group)):
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    pops.append(pop_dict[tid])                    
        avg_pop = sum(pops)/len(pops)
        avg_pops.append(avg_pop)
        
    print(avg_pops)

In [11]:
HitNOVs = []
for lambda_ in tqdm(lambdas):
    top_n_size = 50
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    HitNOV = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]
        
        nov = 0
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0:
                        nov += (1-pop_dict[tid])
        
        HitNOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
    HitNOVs.append(HitNOV)
    
for HitNOV in HitNOVs:
    print(HitNOV)


[0.056168388429752004, 0.058690303030302905, 0.0622683284457477, 0.06392121212121205, 0.06671306818181817, 0.06587012987012977, 0.06784545454545449, 0.07061616161616163, 0.06604659090909094, 0.07303636363636365]
[0.0679979338842974, 0.06806121212121191, 0.06958651026392947, 0.07081645021645011, 0.07103030303030294, 0.07028571428571415, 0.07128749999999988, 0.07459469696969691, 0.0697534090909091, 0.07474545454545456]
[0.0851306818181817, 0.0820872727272726, 0.08041129032258043, 0.08000692640692618, 0.07992897727272723, 0.07843956043956021, 0.07916022727272715, 0.08123484848484842, 0.07602272727272737, 0.07976363636363634]
[0.10091167355371912, 0.09505757575757563, 0.09159090909090897, 0.08983203463203444, 0.08719696969696956, 0.086213786213786, 0.0891613636363635, 0.08914772727272705, 0.08077386363636369, 0.08613181818181817]
[0.11352737603305806, 0.10566969696969675, 0.10267302052785927, 0.09632207792207781, 0.09733428030303008, 0.09218781218781207, 0.09616704545454528, 0.09593560606

In [66]:
diversities = []
novelties = []
for lambda_ in tqdm(lambdas):
    top_n_size = 4
    top_n_items_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_items = pd.DataFrame() 

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]   
            if len(top_n_items) == 0:
                top_n_items = rating_table
            else:
                top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()

        top_n_items_g.append(top_n_items)   
    
    diversity = []
    for i in range(len(groups_n)):
        top_n_items = top_n_items_g[i]
        diversity.append(len(top_n_items)/len(df['tid'].unique()))
    diversities.append(diversity)

    novelty = []
    for i in range(len(groups_n)):
        top_n_items = top_n_items_g[i]
        novelty.append(len(top_n_items[top_n_items['tid'].isin(long_tail)])/len(df['tid'].unique()))
    novelties.append(novelty)
    
for diversity in diversities:
    print(diversity)
print()
for novelty in novelties:
    print(novelty)


[7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05, 7.276828758027252e-05]
[0.00010915243137040878, 0.00010915243137040878, 9.096035947534064e-05, 9.096035947534064e-05, 9.096035947534064e-05, 9.096035947534064e-05, 9.096035947534064e-05, 9.096035947534064e-05, 9.096035947534064e-05, 9.096035947534064e-05]
[0.0003092652222161582, 0.00027288107842602196, 0.0001819207189506813, 0.00016372864705561317, 0.00010915243137040878, 0.0001273445032654769, 0.00010915243137040878, 0.00010915243137040878, 9.096035947534064e-05, 9.096035947534064e-05]
[0.0005457621568520439, 0.0004911859411668394, 0.0004002255816914988, 0.0004002255816914988, 0.00027288107842602196, 0.0002546890065309538, 0.00016372864705561317, 0.00016372864705561317, 0.00016372864705561317, 0.00010915243137040878]
[0.0007276828758027252, 0.0008004511633829977, 0.00061853044443

In [74]:
tid_pd = pd.DataFrame(data={'tid': df['tid'].unique()})
tid_pd = tid_pd.sort_values(by=['tid']).reset_index()
long_tail_list = tid_pd['tid'].isin(long_tail)
long_tail_list = list(long_tail_list)
long_tail_list.count(True)

54420

In [75]:
diversities = []
novelties = []
for lambda_ in tqdm(lambdas):
    top_n_size = 50
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    diversity = []
    novelty = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        
        hits = np.zeros(len(df['tid'].unique()), dtype=bool)
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]
            
            for k in range(len(group)):
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0:
                        hits[int(tid)] = True
        
        diversity.append(list(hits).count(True)/len(df['tid'].unique()))
        novelty.append(list(hits&long_tail_list).count(True)/len(df['tid'].unique()))
    diversities.append(diversity)
    novelties.append(novelty)
    
for diversity in diversities:
    print(diversity)
print()
for novelty in novelties:
    print(novelty)


[0.0009096035947534065, 0.0009096035947534065, 0.0009096035947534065, 0.0009096035947534065, 0.0009096035947534065, 0.0009096035947534065, 0.0009096035947534065, 0.0009096035947534065, 0.0009096035947534065, 0.0009096035947534065]
[0.001546326111080791, 0.001418981607815314, 0.001291637104549837, 0.001273445032654769, 0.0011824846731794284, 0.0011097163855991559, 0.0011097163855991559, 0.0010915243137040878, 0.0011097163855991559, 0.0009459877385435427]
[0.00252869799341447, 0.0021648565555131075, 0.0019465516927722898, 0.001819207189506813, 0.0017464389019265405, 0.0015827102548709273, 0.0015099419672906548, 0.0014917498953955866, 0.0014371736797103823, 0.0010369480980188834]
[0.003656606450908694, 0.003274572941112263, 0.0028925394313158324, 0.002583274209099674, 0.0024559297058341974, 0.0023103931306736524, 0.0021648565555131075, 0.0020011279084574942, 0.0020557041241426985, 0.0011642926012843603]
[0.005021011843038803, 0.004693554548927577, 0.004420673470501555, 0.0038203350979643